# AKNet

In [1]:
import os
import pathlib
import tensorflow as tf
import pandas as pd
import wandb

from bcd.model.network.base import NetworkConfig
from bcd.model.network.aknet import AKNetConfig, AKNetFactory
from bcd.model.repo import ExperimentRepo
from bcd.model.pretrained import *
from bcd.model.experiment import FeatureExtractionExperiment
from bcd.model.config import *

pd.set_option('display.max_rows',999)

## Configuration

In [2]:
# Experiment Parameters
%env "WANDB_NOTEBOOK_NAME" "aknet.ipynb"
mode = "Stage"
force = False  # Whether to retrain if the model and config already exists.
base_models = [DenseNet(), EfficientNet(), Inception(), InceptionResNet(), MobileNet(), ResNet(), Xception()]

env: "WANDB_NOTEBOOK_NAME"="aknet.ipynb"


In [3]:
def create_config(network_config: NetworkConfig):
    project_config = ProjectConfig(mode=mode)

    train_config = TrainConfig(epochs=50, learning_rate=1e-4)    

    dataset_config = DatasetConfig(mode=mode)

    checkpoint_config = CheckPointConfig(monitor="val_accuracy", verbose=1, save_best_only=True, save_weights_only=False, mode="auto")

    early_stop_config = EarlyStopConfig(min_delta=1e-4, monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)

    learning_rate_schedule_config = LearningRateScheduleConfig(min_delta=1e-4, monitor="val_loss", factor=0.5, patience=3, restore_best_weights=True, verbose=1, mode="auto")

    config = Config(project=project_config, 
                    dataset=dataset_config, 
                    train=train_config, 
                    network=network_config, 
                    checkpoint=checkpoint_config, 
                    early_stop=early_stop_config, 
                    learning_rate_schedule=learning_rate_schedule_config)
    return config

network_config = AKNetConfig(activation="sigmoid", input_shape=(224,224,3), output_shape=1, dense1=2048, dropout1=0.5, dense2=1024, dropout2=0.5)
config = create_config(network_config=network_config)

## Load Data

In [4]:
train_dir = pathlib.Path(config.dataset.train_dir).with_suffix('') 
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    seed=123,
    batch_size=config.dataset.batch_size)

# Validation DataSet (10%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    seed=123,
    batch_size=config.dataset.batch_size)

Found 816 files belonging to 2 classes.
Using 653 files for training.
Found 816 files belonging to 2 classes.
Using 163 files for validation.


## Callbacks

In [5]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=config.early_stop.monitor, 
                                                       min_delta=config.early_stop.min_delta,
                                                       patience=config.early_stop.patience, 
                                                       restore_best_weights=config.early_stop.restore_best_weights,
                                                       verbose=config.early_stop.verbose)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor=config.learning_rate_schedule.monitor,
                                                          factor=config.learning_rate_schedule.factor,
                                                          patience=config.learning_rate_schedule.patience,
                                                          verbose=config.learning_rate_schedule.verbose,
                                                          mode=config.learning_rate_schedule.mode,
                                                          min_delta=config.learning_rate_schedule.min_delta,
                                                          min_lr=config.learning_rate_schedule.min_lr)
callbacks = [early_stop_callback, reduce_lr_callback]

## Dependencies

In [6]:
repo = ExperimentRepo(mode = mode, project=config.project.name)
optimizer=tf.keras.optimizers.Adam
metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]

## Build Factory

In [7]:
factory = AKNetFactory(config=config.network)

## Run Experiments

In [8]:
for base_model in base_models:
    network = factory.create(base_model=base_model)
    experiment = FeatureExtractionExperiment(network=network, config=config, optimizer=optimizer, repo=repo, callbacks=callbacks, metrics=metrics, tags=[network.architecture, base_model.name], force=False)
    experiment.run(train_ds=train_ds, val_ds=val_ds)


# ================================================================================================ #
                                           AKNet_DenseNet                                           
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 AKNet_DenseNet_input_layer   [(None, 224, 224, 3)]    0         
 (InputLayer)                                                    
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 tf.math.truediv_1 (TF

wandb: Currently logged in as: aistudio. Use `wandb login --relogin` to force relogin


Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 1.7916 - accuracy: 0.5008 - auc: 0.5063 - precision: 0.4306 - recall: 0.4336
Epoch 1: val_accuracy improved from -inf to 0.60736, saving model to models/stage/AKNet_EfficientNet-vij64dxe.keras
21/21 [==============================] - 112s 4s/step - loss: 1.7916 - accuracy: 0.5008 - auc: 0.5063 - precision: 0.4306 - recall: 0.4336 - val_loss: 1.6488 - val_accuracy: 0.6074 - val_auc: 0.6120 - val_precision: 0.3750 - val_recall: 0.0484 - lr: 1.0000e-04
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 1.8183 - accuracy: 0.5942 - auc: 0.6048 - precision: 0.5385 - recall: 0.5140
Epoch 2: val_accuracy did not improve from 0.60736
21/21 [==============================] - 89s 4s/step - loss: 1.8183 - accuracy: 0.5942 - auc: 0.6048 - precision: 0.5385 - recall: 0.5140 - val_loss: 1.1523 - val_accuracy: 0.5890 - val_auc: 0.5880 - val_precision: 0.4684 - val_recall: 0.5968 - lr: 1.0000e-04
Epoch 3/50
21/21 [====

wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch/accuracy,▁▃▃▄▄▅▅▄▅▆▇▆▇███
epoch/auc,▁▃▃▃▄▅▅▅▅▆▆▇▇███
epoch/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
epoch/learning_rate,████████▄▄▄▂▂▂▁▁
epoch/loss,██▇▇▅▄▄▅▄▃▂▂▂▁▁▁
epoch/lr,█████████▄▄▄▂▂▂▁
epoch/precision,▁▃▃▄▄▅▅▄▅▆▇▆▇███
epoch/recall,▁▃▄▃▅▄▄▄▆▇▆▆▇██▇
epoch/val_accuracy,▇▆▁▆█▇▇▁▄▇▅▅▇█▆▆
epoch/val_auc,▃▁▃▅▆█▅▁▄▃▅▅▄▅▅▆
epoch/val_loss,█▄▃▂▂▁▂▄▃▃▂▂▂▂▂▂


...

epoch/accuracy,▁▁▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇██████████████
epoch/auc,▁▁▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
epoch/learning_rate,██████████████████▄▄▄▄▄▄▄▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/loss,██▆▆▆▆▆▆▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/lr,████████████████████▄▄▄▄▄▄▄▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/precision,▁▁▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇███▇▇██▇██▇▇██
epoch/recall,▁▁▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▇▇▆▆▇▇▇████▇▇█▇▇████
epoch/val_accuracy,▁▁▇▇▄▄▄▄▃▄▄▇▇▅▅▂▂█▅▅▇▇▅▅▆▆▅██▇▇▆▆▇▇▇██▇▇
epoch/val_auc,▁▁▄▄▅▅▅▅▆▆▆▇▇████▇▆▆▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇
epoch/val_loss,▇▇▃▃▅▅▆▆▃▆▆▁▁▁▁██▂▂▂▁▁▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂


# ================================================================================================ #
                                           AKNet_Xception                                           
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 AKNet_Xception_input_layer   [(None, 224, 224, 3)]    0         
 (InputLayer)                                                    
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                      

Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 1.7251 - accuracy: 0.4977 - auc: 0.5349 - precision: 0.4466 - recall: 0.4569
Epoch 1: val_accuracy did not improve from 0.69325

Epoch 1: val_accuracy did not improve from 0.69325

Epoch 1: val_accuracy did not improve from 0.69325

Epoch 1: val_accuracy did not improve from 0.68098

Epoch 1: val_accuracy did not improve from 0.66871

Epoch 1: val_accuracy improved from -inf to 0.59509, saving model to models/stage/AKNet_Xception-vaxm0d04.keras
21/21 [==============================] - 109s 5s/step - loss: 1.7251 - accuracy: 0.4977 - auc: 0.5349 - precision: 0.4466 - recall: 0.4569 - val_loss: 0.7543 - val_accuracy: 0.5951 - val_auc: 0.6098 - val_precision: 0.4714 - val_recall: 0.5323 - lr: 1.0000e-04
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 1.6365 - accuracy: 0.5620 - auc: 0.5591 - precision: 0.5000 - recall: 0.4965
Epoch 2: val_accuracy did not improve from 0.69325

Epoch 2: val_accuracy did

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch/accuracy,▁▁▁▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▇▇▅▅▅▇▇▇▇███▇▇
epoch/auc,▁▁▁▂▂▃▃▃▃▃▃▃▄▄▃▃▃▃▃▅▅▅▅▅▆▆▇▇▆▆▆▇▇▇▇███▇▇
epoch/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇█████
epoch/learning_rate,███████▄▄▄▄▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,███▇▇▆▆▅▅▅▄▄▄▄▄▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
epoch/lr,██████████▄▄▄▄▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/precision,▁▁▁▂▂▃▃▄▄▄▄▄▄▄▄▄▄▃▃▅▅▆▆▆▆▆▇▇▅▅▅▇▇▇▇███▇▇
epoch/recall,▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▅▅▇▇▅▅▅▇▇▇▇█████
epoch/val_accuracy,▆▆▆██▃▃███▄▄▁▁▆▆▆▆▆▅▅▄▄▄▇▇▅▅▆▆▆▅▅▆▆▇▇▇▅▅
epoch/val_auc,▅▅▅▂▂▁▁▁▁▁▅▅▅▅▅▅▅▆▆▅▅▄▄▄▅▅▅▅▇▇▇▇▇███████
epoch/val_loss,▃▃▃▇▇▄▄███▃▃▅▅▁▁▁▁▁▁▁▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁
